TODO (SJ):
1. Add the pickle creation stuff.

In [1]:
import os, sys
import csv
from collections import defaultdict
from datetime import datetime
import json

sys.path.insert(0, "/userdata/sjain/ECoG_data_collection/")
from ecog_preprocessing.transformData import transform as transformData

In [ ]:
# mkdir data/processed
datapath = "/data_store1/human/prcsd_data/"
savepath = "data/processed"

subj_blocks = defaultdict(list)
bad_combos = []
good_rows = []

with open("task_database_logging/task_database_Info_20240313.csv", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = [row for row in reader]
    
if os.path.exists("task_database_logging/task_database_Log.json"):
    with open("task_database_logging/task_database_Log.json", "r") as f:
        LOAD = json.load(f)
    done_blocks = [f"{db['SubjectID']}_B{db['Block']}" for db in LOAD]
# Read each entry from our task database.
for row in rows:
    subj, block, task, valid = row["SubjectID"], row["Block"], row["Task"], row["Valid"]
    if valid.lower()=="bad": continue
    subj_blocks[subj].append(block)

    blockname = f"{subj}_B{block}"
    blockpath = os.path.join(datapath, subj, blockname)
    storepath = os.path.join(savepath, subj, blockname)
    
    if blockname in done_blocks:
        print(f"{blockname} was processed already!")
        continue
    try:
        print(f"Processing blockname {blockname}.\n")
        transformData(blockpath, rate=400, car=False, line_noise_notch=True, force=True, storepath=storepath)
        # If able to preprocess, record the entire `row` as good.
        row["prcsd_date"] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        row["prcsd_loc"] = storepath
        good_rows.append(row)
        with open("task_database_logging/task_database_Log.json", "w") as f:
            json.dump(good_rows, f)
    except FileNotFoundError:
        print(f"Didn't find data. Skipping blockname {blockname}.\n")
        # If not able to preprocess, record the `subject-block` as bad.
        bad_combos.append(blockname)

Processing blockname EC6_B36.

do downsampling
Didn't find data. Skipping blockname EC6_B36.

Processing blockname EC6_B38.

do downsampling
Didn't find data. Skipping blockname EC6_B38.

Processing blockname EC6_B42.

do downsampling
Didn't find data. Skipping blockname EC6_B42.

EC28_B3 was processed already!
Processing blockname EC28_B4.

do downsampling
Didn't find data. Skipping blockname EC28_B4.

EC28_B6 was processed already!
EC28_B7 was processed already!
EC28_B13 was processed already!
EC28_B19 was processed already!
EC28_B21 was processed already!
EC28_B23 was processed already!
EC28_B28 was processed already!
EC28_B30 was processed already!
EC28_B36 was processed already!
EC28_B40 was processed already!
EC34_B5 was processed already!
EC34_B10 was processed already!
EC34_B14 was processed already!
EC34_B17 was processed already!
EC34_B40 was processed already!
EC34_B44 was processed already!
EC34_B47 was processed already!
EC34_B51 was processed already!
Processing blockname

In [ ]:
bad_combos